 aqui se preparan los conjuntos de datos necesarios para hacer las consultas en la API.

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq



+ def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}


+def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]} 

+ def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

+def UsersWorstDeveloper( año : int ): Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

+def sentiment_analysis( empresa desarrolladora : str ): Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.
Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}



In [9]:

df_games = pd.read_parquet(r'./Data/clean_games_steam.parquet.gzip')
df_items = pd.read_parquet(r'./Data/clean_games_items.parquet.gzip') 
df_reviews = pd.read_parquet(r'./Data/clean_games_reviews.parquet.gzip') 


primer endpoint
Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora

In [ ]:
price_anio_developer = df_games[['price', 'year', 'developer', 'id']]
# Se renombra la columna 'id' para unirla con el dataframe anterior
price_anio_developer = price_anio_developer.rename(columns={'id':'item_id'})
# se eliminan los duplicados
df_items_developer = price_anio_developer.drop_duplicates()
# Reseteamos el indice
df_items_developer


In [ ]:
df_items_developer.to_parquet(r'./Data/endpoint1.parquet.gzip', compression='gzip') # Exportamos


segundo endpoint
Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [ ]:

df_reviews2 = df_reviews[['reviews_recommend']]


In [ ]:
# Se extraen las columnas 'items_count', 'user_id' e 'item_id'
df_gastos_items = df_items[['items_count', 'user_id', 'item_id']]
df_gastos_items


In [ ]:
price_juegos = df_games[['price', 'id']]
# Elimina los duplicados
price_juegos = price_juegos.drop_duplicates(subset='id', keep='first')
# Se renombra 'id' por 'item_id' para unir mas adelante
price_juegos = price_juegos.rename(columns={'id':'item_id'})
price_juegos


In [ ]:
df_gastos_items = pd.concat([df_gastos_items ,price_juegos,df_reviews2], axis=0)
df_gastos_items


In [ ]:
df_gastos_items[df_gastos_items['price'].isnull()]


In [ ]:
df_relleno = df_gastos_items['price'].fillna(0.0)
# Se borra la columna original y se concatena la columna rellena con todo el dataframe
df_gastos_items = pd.concat([df_gastos_items.drop('price', axis=1), df_relleno ], axis=1)
df_gastos_items


In [ ]:
# Se elimina 'item_id'
df_gastos_items = df_gastos_items.drop('item_id', axis=1)
df_gastos_items.columns


In [ ]:
df_gastos_items_group = df_gastos_items.groupby('user_id')['price'].sum().reset_index()
df_gastos_items_group


In [ ]:
df_count_items = df_gastos_items[['items_count', 'user_id']]
# se eliminan los duplicados
df_count_items = df_count_items.drop_duplicates(subset='user_id', keep='first')
df_count_items


In [ ]:
df_gastos_items = df_count_items.merge(df_gastos_items_group, on='user_id', how='right')
df_gastos_items


In [ ]:
df_gastos_items.to_parquet(r'./Data/endpoint2.parquet.gzip', compression='gzip') # Exportamos


endpoint 3 UserForGenre 
Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año. Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [13]:
anio_user = df_games[[ 'year']]


In [34]:
df_UserForGenre = df_items[['user_id','playtime_forever','genre']]
df_UserForGenre  = pd.concat([anio_user, df_UserForGenre], axis=0, ignore_index=True)
# df_UserForGenre.dropna()
df_UserForGenre.dropna(thresh=2)
df_UserForGenre.head()
df_UserForGenre.dropna(subset=['user_id','playtime_forever','genre'])


,year,user_id,playtime_forever,genre
58447,NaN,76561197970982479,6.0,6.0
58448,NaN,76561197970982479,0.0,0.0
58449,NaN,76561197970982479,7.0,7.0
58450,NaN,76561197970982479,0.0,0.0
58451,NaN,76561197970982479,0.0,0.0
...,...,...,...,...
5211651,NaN,76561198329548331,0.0,0.0
5211652,NaN,76561198329548331,0.0,0.0
5211653,NaN,76561198329548331,3.0,3.0
5211654,NaN,76561198329548331,4.0,4.0


In [35]:
df_UserForGenre.to_parquet(r'./Data/endpoint3.parquet.gzip', compression='gzip') # Exportamos



endpoint 4 - best_developer_year
 Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
    Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [11]:
df_best_developer_year = df_reviews[['reviews_recommend','reviews_helpful','reviews_review']]


In [12]:
df_best_developer_year.to_parquet(r'./Data/endpoint4.parquet.gzip', compression='gzip') # Exportamos


endpoint 5 developer_reviews_analysis
 Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo

 El objetivo es asignar un valor numérico a un texto, en este caso a los comentarios que los usuarios dejaron para un juego determinado, para representar si el sentimiento expresado en el texto es negativo, neutral o positivo."

In [3]:
from textblob import TextBlob
import re


In [4]:
def functionsentiment_analisis(review):
    '''
    Realiza un análisis de sentimiento en un texto dado y devuelve un valor numérico que representa el sentimiento.

    Esta función utiliza la librería TextBlob para analizar el sentimiento en un texto dado y
    asigna un valor numérico de acuerdo a la polaridad del sentimiento.

    Parameters:
        review (str): El texto que se va a analizar para determinar su sentimiento.

    Returns:
        int: Un valor numérico que representa el sentimiento del texto:
             - 0 para sentimiento negativo.
             - 1 para sentimiento neutral o no clasificable.
             - 2 para sentimiento positivo.
    '''
    if review is None:
        return 1
    analysis = TextBlob(review)
    polarity = analysis.sentiment.polarity
    if polarity < -0.2:
        return 0  
    elif polarity > 0.2: 
        return 2 
    else:
        return 1 


In [5]:

df_reviews['sentiment_analysis'] = df_reviews['reviews_review'].apply(functionsentiment_analisis)
df_reviews.head()




,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,1
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,1
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,2
3,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2
4,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,Git gud,1


In [6]:
df_developer_reviews_analysis = df_reviews.drop(columns=['reviews_review'])
df_developer_reviews_analysis.columns


Index(['user_id', 'user_url', 'reviews_item_id', 'reviews_helpful',
       'reviews_recommend', 'sentiment_analysis'],
      dtype='object')

In [7]:
df_developer_reviews_analysis.to_parquet(r'./Data/endpoint5.parquet.gzip', compression='gzip') # Exportamos


enpoint 6 
Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado

In [ ]:
df_recomendacion_juego = df_games[['app_name', 'id','genres']]
df_recomendacion_juego


In [ ]:
df_recomendacion_juego.to_parquet(r'./Data/endpoint6.parquet.gzip', compression='gzip') # Exportamos
